In [4]:
## Import Necessary Libraries
import pprint
from fastcore.basics import patch
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipycytoscape
from ipycytoscape import CytoscapeWidget

## Creating Author Class
class Author:
    def __init__(self, first, middle, last, email=None, publications=[]):
        self.first = first
        self.middle = middle
        self.last = last
        self.email = email
        self.publications = publications

## Make format pretty
@patch
def __repr__(self:Author):
    return pprint.pformat(vars(self))

## Mock Dataset
a1 = Author('A', 'B', 'Carlson', publications=[1, 2, 3])
a2 = Author('B', 'C', 'Dawson', publications=[3, 4, 5, 6])
a3 = Author('C', 'D', 'Elfson', publications=[1, 5, 7, 8])
a4 = Author('D', 'E', 'Fitzgerald', publications=[1, 5, 9, 10])
a5 = Author('E', 'F', 'Gerard', publications=[4, 11, 12])
a5

authors = [a1, a2, a3, a4, a5]

#### Step 1: Making sure correct matches are found between authors ####

# Create the graph
G = nx.Graph()

for index_a in range(len(authors)):
    for index_b in range(index_a + 1, len(authors)):
        author_a = authors[index_a]
        author_b = authors[index_b]

        common_publications = set(author_a.publications).intersection(author_b.publications)
        if common_publications:
            # Add the weighted edge to the graph
            G.add_edge(author_a, author_b, weight=len(common_publications))
            print(f"Match found between {author_a.first} {author_a.last} and {author_b.first} {author_b.last}")
            print(f"Matching Publications: {common_publications}")
            print(f"Weighted Edge: {author_a.first} {author_a.last} - {author_b.first} {author_b.last} (Weight: {len(common_publications)})")

# Create the Cytoscape widget
cyto = CytoscapeWidget()

# Set node size based on the weight of the edges
min_weight = min(G.edges(data=True), key=lambda x: x[2]['weight'])[2]['weight']
max_weight = max(G.edges(data=True), key=lambda x: x[2]['weight'])[2]['weight']

for node, data in G.nodes(data=True):
    weight = sum(data.get('weight', 0) for _, _, data in G.edges(node, data=True))
    node_size = 30 + 20 * ((weight - min_weight) / (max_weight - min_weight))  # Adjust the size range as desired
    data['size'] = node_size

# Add nodes and edges to the Cytoscape graph
cyto.graph.add_graph_from_networkx(G)

# Create an output widget for displaying author data
output = widgets.Output()

# Callback function for handling node click event
def on_node_click(event):
    with output:
        output.clear_output()
        if 'name' in event['data']:
            # Get the author name from the clicked node
            author_name = event['data']['name']
            # Find the corresponding Author object
           # author = next((a for a in authors if a.__repr__() == author_name), None)
            if author:
                # Display author data in the output widget
                print(f"Author: {author.first} {author.middle} {author.last}")
                print(f"Email: {author.email or 'N/A'}")
                print(f"Publications: {str(author.publications)}")

# Set the callback function for node click event
cyto.on('node', 'click', on_node_click)

# Create a label for the title
title_label = widgets.Label(value='Author Network Graph', style={'font-weight': 'bold'})

# Display the widgets using a VBox layout
widgets.VBox([title_label, widgets.HBox([cyto, output])])


Match found between A Carlson and B Dawson
Matching Publications: {3}
Weighted Edge: A Carlson - B Dawson (Weight: 1)
Match found between A Carlson and C Elfson
Matching Publications: {1}
Weighted Edge: A Carlson - C Elfson (Weight: 1)
Match found between A Carlson and D Fitzgerald
Matching Publications: {1}
Weighted Edge: A Carlson - D Fitzgerald (Weight: 1)
Match found between B Dawson and C Elfson
Matching Publications: {5}
Weighted Edge: B Dawson - C Elfson (Weight: 1)
Match found between B Dawson and D Fitzgerald
Matching Publications: {5}
Weighted Edge: B Dawson - D Fitzgerald (Weight: 1)
Match found between B Dawson and E Gerard
Matching Publications: {4}
Weighted Edge: B Dawson - E Gerard (Weight: 1)
Match found between C Elfson and D Fitzgerald
Matching Publications: {1, 5}
Weighted Edge: C Elfson - D Fitzgerald (Weight: 2)
